In [8]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain.retrievers import WikipediaRetriever

llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o-mini-2024-07-18",
)


def save_the_file(docs, title):
    f = open(f"./{title}.txt", "w", encoding="utf-8")
    f.write(docs)


def serach_wikipedia(keyword):
    rs = "Wikipedia Result\n\n"
    retriver = WikipediaRetriever(top_k_results=3, lang="ko")
    data_list = retriver.get_relevant_documents(keyword)
    for page_content in data_list:
        rs += f"{page_content.page_content} \n\n"
    return rs


class WikipediaToolArgsSchema(BaseModel):
    keyword: str = Field(description="Search from Wikipedia using keywords")


class WikipediaTool(BaseTool):
    name = "Wikipedia"
    description = """
    A tool used by Wikipedia to search for keywords. Use 'keyword' as a parameter.
    """
    args_schema: Type[WikipediaToolArgsSchema] = WikipediaToolArgsSchema

    def _run(self, keyword):
        wiki_rs = serach_wikipedia(keyword)
        save_the_file(wiki_rs, "wiki")
        return wiki_rs


class DuckDuckGoToolArgsCshema(BaseModel):
    query: str = Field(description="Search from DuckDuckGo using keywords")


class DuckDuckGoSearchTool(BaseTool):
    name = "DuckDuckGoSearchTool"
    description = """
    A tool used by DuckDuckGo to search for keywords. Use 'keyword' as a parameter.    
    """
    args_schema: Type[DuckDuckGoToolArgsCshema] = DuckDuckGoToolArgsCshema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        ddg_rs = ddg.run(query)
        save_the_file(ddg_rs, "ddg")
        return ddg_rs


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaTool(),
    ],
)

prompt = "Research about the XZ backdoor"

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `Wikipedia` with `{'keyword': 'XZ backdoor'}`


Wikipedia Result

CVE-2024-3094는 2024년 3월 29일 소프트웨어 개발자인 안드레스 프런드가 발견한 2024년 2월 liblzma 라이브러리 내 Linux 유틸리티 xz (버전 5.6.0 및 5.6.1)에 악의적으로 삽입된 백도어이다. xz는 대부분의 리눅스 배포판에 기본적으로 설치되어 있지만 백도어는 x86-64 아키텍처에서 실행되는 데비안 및 RPM 기반 시스템만을 표적으로 삼았다. 발견 당시 백도어가 포함된 소프트웨어 버전은 아직 널리 배포되지 않은 상태였다.
백도어는 영향을 받는 리눅스 시스템에서 특정 Ed448 개인 키 원격 코드 실행를 보유한 공격자에게 원격 코드 실행 권한을 제공한다. 이 문제에는 소프트웨어 취약점에 부여되는 점수 중 가장 높은 점수인 10.0의 CVSS 점수가 할당되었다.


== 배경 ==
마이크로소프트 직원이자 PostgreSQL 개발자인 안드레스 프런드는 데비안 Sid의 성능 회귀에 대해 조사한 후 백도어를 보고했다. 프런드는 SSH 연결로 인해 예기치 않게 많은 양의 CPU 사용량이 발생하고 메모리 디버깅 도구인 Valgrind에서 오류가 발생한다는 사실을 발견했다. 프런드는 자신이 발견한 내용을 오픈월 프로젝트의 오픈 소스 보안 메일링 리스트에 보고했으며, 이는 다양한 소프트웨어 공급업체의 관심을 끌었다. 백도어는 여러 단계로 실행되도록 구성된 난독화를 통해 숨겨져 있었다.
백도어가 삽입된 버전이 운영 체제에 설치되면 systemd 라이브러리를 남용하여 오픈SSH의 SSH 서버 데몬의 동작을 변경하여 공격자가 인증된 관리자와 동일한 수준의 액세스 권한을 얻을 수 있게 만든다. 레드햇의 분석에 따르면 백도어는 악의적인 행위자가 sshd 인증을 깨고 원격으로 전체 시스템에 대한 무단 액세스를 얻을 수 있도록 만들어준다.
후속

{'input': 'Research about the XZ backdoor',
 'output': 'The XZ backdoor, identified as CVE-2024-3094, was discovered on March 29, 2024, by software developer Andres Freund. It was found in the liblzma library within the Linux utility xz (versions 5.6.0 and 5.6.1), which is commonly installed on most Linux distributions. The backdoor specifically targeted Debian and RPM-based systems running on the x86-64 architecture.\n\n### Background\nAndres Freund reported the backdoor after investigating performance regressions in Debian Sid. He noticed unexpected high CPU usage due to SSH connections and errors in the memory debugging tool Valgrind. His findings were shared on the open-source security mailing list, drawing attention from various software vendors. The backdoor was cleverly concealed through multiple layers of obfuscation.\n\nWhen the compromised version of xz was installed on a system, it exploited the systemd library to alter the behavior of the OpenSSH SSH server daemon, allowing

In [9]:
agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        DuckDuckGoSearchTool(),
    ],
)

prompt = "Research about the XZ backdoor"

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `DuckDuckGoSearchTool` with `{'query': 'XZ backdoor'}`


What Does the Backdoor Do? Malicious code added to xz Utils versions 5.6.0 and 5.6.1 modified the way the software functions. The backdoor manipulated sshd, the executable file used to make remote ... That operation matches the style of the XZ Utils backdoor far more than the cruder supply chain attacks of APT41 or Lazarus, by comparison. "It could very well be someone else," says Aitel. A malicious code inserted into a Linux compression tool by a developer named JiaT75 could have compromised millions of systems via SSH. A Microsoft worker discovered the backdoor and alerted the open source community, preventing a major disaster. A backdoor was planted in the XZ utils package, a popular open-source compression utility for Linux/UNIX systems, by a threat actor named JiaT75. The backdoor exploited the SSH protocol to gain remote access and control over affected devices. The xz-util

{'input': 'Research about the XZ backdoor',
 'output': 'The XZ backdoor refers to a malicious code that was inserted into the XZ Utils package, specifically in versions 5.6.0 and 5.6.1. XZ Utils is a widely used open-source compression utility for Linux and UNIX systems. The backdoor was introduced by a threat actor identified as JiaT75.\n\n### Key Points about the XZ Backdoor:\n\n1. **Functionality**: The backdoor manipulated the SSH daemon (sshd), which is the executable file used for remote access to systems. This allowed the attacker to gain unauthorized remote access and control over affected devices.\n\n2. **Discovery**: The backdoor was discovered by a Microsoft employee who alerted the open-source community, which helped prevent a potentially widespread compromise of systems.\n\n3. **Impact**: Given the popularity of the XZ Utils package among various Linux distributions, the backdoor had the potential to compromise millions of systems through SSH authentication.\n\n4. **Respon